In [1]:
import elasticsearch

In [2]:
import pprint

In [3]:
app_qry = \
{
  "size": 0,
  "aggs": {
    "provider_filter": {
      "filter": {"term": {"provider_code": "aws"}},
      "aggs": {
        "account_filter": {
            "filter": {"term": {"provider_account": "123"}}
            
        }
      }
    }
  }
}

## Group by provider without context filter 

In [6]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])
query_by_app = \
{
  "size": 0,
  "aggs": {
    "provider_filter": {"terms": {"field": "provider_code"}}
  }
}
import time
start_time = time.time()
res = es.search(index="cloud_resources_five_million", body=query_by_app)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.8274548053741455 seconds ---


In [7]:
res

{'took': 761,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'provider_filter': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'aws', 'doc_count': 4000003},
    {'key': 'testprovider', 'doc_count': 1000854}]}}}

## Groupby provider with context filter 

In [8]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])
query_by_app = \
{
  "size": 0,
  "aggs": {
    "provider_filter": {
        "terms": {"field": "provider_code"},
        "aggs": {
            "account_filter": {
                "terms": {"field": "provider_account"},
                "aggs": {
                    "application_filter": {
                        "terms": {"field": "context.application"}
                    }
                }
            
             }
        }
    }
  }
}
import time
start_time = time.time()
res = es.search(index="cloud_resources_five_million", body=query_by_app)
print("--- %s seconds ---" % (time.time() - start_time))

--- 2.855454921722412 seconds ---


In [9]:
res

{'took': 2833,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'provider_filter': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'aws',
     'doc_count': 4000003,
     'account_filter': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': '123',
        'doc_count': 1000001,
        'application_filter': {'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 284307,
         'buckets': [{'key': 'aiops', 'doc_count': 72217},
          {'key': 'store_aiops', 'doc_count': 71836},
          {'key': 'd2ops', 'doc_count': 71803},
          {'key': 'ecommerce_app', 'doc_count': 71789},
          {'key': 'd2ops_aiops', 'doc_count': 71529},
          {'key': 'cloud_based_app', 'doc_count': 71494},
          {'key': 'aiops_aiops', 'doc_count': 71346},
   

## Group by context app and env

In [22]:
PUT cloud_resources/_mapping
{
  "properties": {
    "context.environment": { 
      "type":     "text",
      "fielddata": true
    }
  }
}

SyntaxError: invalid syntax (<ipython-input-22-9599a5828961>, line 1)

In [11]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])
query_by_app = \
{
  "size": 0,
  "aggs": {
    "provider_filter": {
        "terms": {"field": "provider_code"},
        "aggs": {
            "account_filter": {
                "terms": {"field": "provider_account"},
                "aggs": {
                    "application_filter": {
                        "terms": {"field": "context.application"},
                        "aggs":{
                            "environment_filter": {
                                "terms": {"field": "context.environment"}
                            }
                        }
                    }
                }
            
             }
        }
    }
  }
}
import time
start_time = time.time()
res = es.search(index="cloud_resources", body=query_by_app)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.025203227996826172 seconds ---


In [24]:
res

{'took': 2251,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'provider_filter': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'aws',
     'doc_count': 800000,
     'account_filter': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': '123',
        'doc_count': 200000,
        'application_filter': {'doc_count_error_upper_bound': 0,
         'sum_other_doc_count': 0,
         'buckets': [{'key': 'finance_app',
           'doc_count': 29002,
           'environment_filter': {'doc_count_error_upper_bound': 0,
            'sum_other_doc_count': 0,
            'buckets': [{'key': 'dev', 'doc_count': 9815},
             {'key': 'qa', 'doc_count': 9622},
             {'key': 'uat', 'doc_count': 9565}]}},
          {'key': 'd2ops',
           'doc_count': 2878

In [12]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])

query = {
  "size": 0,
#   "query": {
#     "bool": {
#       "must": [
#         { "match": { "provider_code": "TestProvider"}
#         }
#       ],
#       "filter": [
#         { "terms":  { "context.application": ["cam", "aiops"] }},
#         { "terms":  { "context.environment": ["dev", "uat"] }}
#       ]
#     }
#   },
  "aggs" : {
    "data": {
      "composite" : {
        "size": 10000,
        "sources": [
          {"provider_code": {"terms": {"field": "provider_code.keyword"}}},
          {"type": { "terms": {"field": "type.keyword" } } },
          {"app": { "terms": {"field": "context.application.keyword" } } },
          {"env": { "terms": {"field": "context.environment.keyword" } } }
          ]
        }
    }
  }
}
import time
start_time = time.time()
res = es.search(index="cloud_resources", body=query)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.03855299949645996 seconds ---


In [28]:
res


{'took': 1631,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'data': {'after_key': {'provider_code': 'aws',
    'type': 'service_instance',
    'app': 'store',
    'env': 'uat'},
   'buckets': [{'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'aiops',
      'env': 'dev'},
     'doc_count': 5531},
    {'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'aiops',
      'env': 'qa'},
     'doc_count': 5325},
    {'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'aiops',
      'env': 'uat'},
     'doc_count': 5319},
    {'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'cam',
      'env': 'dev'},
     'doc_count': 4994},
    {'key': {'provider_code': 'TestProvider',
    

In [40]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])

query = {
  "size": 0,
  "query": {
    "bool": {
      "must": [
        { "match": { "provider_code": "TestProvider"}
        }
      ],
      "filter": [
        { "terms":  { "context.application": ["cam", "aiops"] }},
        { "terms":  { "context.environment": ["dev", "uat"] }}
      ]
    }
  },
  "aggs" : {
    "data": {
      "composite" : {
        "size": 10000,
        "sources": [
          {"provider_code": {"terms": {"field": "provider_code.keyword"}}},
          {"type": { "terms": {"field": "type.keyword" } } },
          {"app": { "terms": {"field": "context.application.keyword" } } },
          {"env": { "terms": {"field": "context.environment.keyword" } } }
          ]
        }
    }
  }
}
import time
start_time = time.time()
res = es.search(index="cloud_resources", body=query)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.02229619026184082 seconds ---


In [37]:
res


{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'data': {'after_key': {'provider_code': 'TestProvider',
    'type': 'service_instance',
    'app': 'cam',
    'env': 'uat'},
   'buckets': [{'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'aiops',
      'env': 'dev'},
     'doc_count': 5531},
    {'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'aiops',
      'env': 'uat'},
     'doc_count': 5319},
    {'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'cam',
      'env': 'dev'},
     'doc_count': 4994},
    {'key': {'provider_code': 'TestProvider',
      'type': 'associated_component',
      'app': 'cam',
      'env': 'uat'},
     'doc_count': 5483},
    {'key': {'provider_code': 'TestProvider',
 

## Sorting on elastic search 

In [46]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

es = Elasticsearch(["http://localhost:9200"])


In [52]:
query = {
  "size": 10,
    "query": {
    "bool": {
      "must": [
        { "match": { "provider_code": "TestProvider"}
        }
      ],
      "filter": [
        { "terms":  { "context.application": ["cam", "aiops"] }},
        { "terms":  { "context.environment": ["dev", "uat"] }}
      ]
    }
  }
}

import time
start_time = time.time()
res = es.search(index="cloud_resources", body=query)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.028666019439697266 seconds ---


In [49]:
res

{'took': 211,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 1.6094363,
  'hits': [{'_index': 'cloud_resources',
    '_type': 'inventory',
    '_id': 'Br4GUGsB9FCRiI2zFw4p',
    '_score': 1.6094363,
    '_source': {'provider_account': '12345678',
     'provider_code': 'TestProvider',
     'provider_resource_id': {'value': '2RRURV0EMLOAXC2EGVHAPZOH8BTIPWLKFRBUB1XXUXBAQ',
      'name': 'DJC3QJH6BP29CL2X22H3NTAFFOT40SOSZCHL4YQFYYL2CCY2XM8WG68OZHMKERC28'},
     'provider_resource_name': 'RDS - bdf0e52e-7a03-486b-9693-18f032d22a21',
     'status': 'Active',
     'manage': 'ibm_managed',
     'service_category_type': 'database',
     'type': 'independent_component',
     'provider_account_name': 'test_account_1',
     'location': {'region_code': 'eu-central-1',
      'availability_zone_code': ['eu-central-1a',
       'eu-central-1b',
       'eu-central-1c']},
     'context': {'a

In [ ]:
query = {
  "size": 10,
    "query": {
    "bool": {
      "must": [
        { "match": { "provider_code": "TestProvider"}
        }
      ],
      "filter": [
        { "terms":  { "context.application": ["cam", "aiops"] }},
        { "terms":  { "context.environment": ["dev", "uat"] }}
      ]
    }
  }
}

import time
start_time = time.time()
res = es.search(index="cloud_resources", body=query)
print("--- %s seconds ---" % (time.time() - start_time))